In [1]:
#!/usr/bin/env python
# coding: utf-8

###========================================================###
###========================================================###
###         IDENTITY-BASED LINKABLE RING SIGNATURE         ###
###                   (TRAPDOOR MECHANISM)                 ###
###========================================================###
###========================================================###

from timeit import default_timer as timer
import sys
import random
import time
import hashlib
import statistics
import os
import matplotlib.pyplot as plt 
import time
from sage.all import *  # import sage library
import gc
def gauss_function_number(x, c, s):
    """
    * Input:
        - x: a variable
        - c: a center
        - s: a Gaussian paramter
    * Output:
        - exp(-pi*(x-c)**2/s**2)
    """
    #If x,c are real numbers
   
    return exp(-pi*(x-c)**2/(s**2))
   



#=================
def gauss_function_vector(xx, cc, s):
    """
    * Input
        - a variable x
        - a center c
        - a Gaussian paramter s
    * Output:
        - exp(-pi*(x-c)**2/s**2)
    """
    #If x,c are real numbers
    return exp(-pi*(xx-cc).norm()**2/(s**2))


#=======================================================
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
def sample_z(s,c):
    """
    Sample from D_{ZZ,s,c} defined by \rho_s,c(x):=exp(-(x-c)**2/(2*s**2))
    s: Gaussian parameter
    c: center
    Input:
    
    Output:
    
    See at https://doc.sagemath.org/html/en/reference/stats/sage/stats/distributions/discrete_gaussian_integer.html
    """
    D=DiscreteGaussianDistributionIntegerSampler(s, c, algorithm="sigma2+logtable")
    return D()

#=======================================================
from sage.stats.distributions.discrete_gaussian_lattice import DiscreteGaussianDistributionLatticeSampler

def sample_d(BB,s,cc):
    """
    This algorithm is to sample a lattice vector from D_{L(BB),s,cc}, defined by \rho_s,c(x):=exp(-||x-c||**2/(2s**2))
    * Input:
        - BB: a basis for the lattice L(BB) (BB can be input as ZZ**m)
        - s: Gaussian parameter
        - cc: center vector
    * Output:
        - A lattice vector sampled from discrete Gaussian D_{L(BB),s,cc}
     See at https://doc.sagemath.org/html/en/reference/stats/sage/stats/
     distributions/discrete_gaussian_lattice.html

    """
    
    D=DiscreteGaussianDistributionLatticeSampler(BB, s, cc)
    return D()


#==========================================================

def gadget_vector(k):
    """
    This algorithm is to generate the gadget vector gg=(1 2 4 8....2**{k-1}) via [MP12]
    * Input: 
        - k: dimension of the output vector gg
    * Output:  
        - gg=(1 2 4 8....2**{k-1}).
    """
    return vector([2**i for i in range(k)])

#===========================================================

def gadget_matrix(n, q):
    """
    This algorithm is to generate the gadget matrix GG via [MP12]

                             |gg  0   0  ...   0 |
                             |0   gg  0  ...   0 |
          Gadget matrix GG=  |0   0   gg ...   0 |    
                             |...................|
                             |0   0    0 ...   gg|
                
    Input: 
        - n: number of rows of GG
        - q: modulus
    
    Output:                 
        - The gadget matrix GG.
    """
  
 
    k=ceil(log(q,2))
    m=n*k
    ZZq = IntegerModRing(q)
    #vector gg=[1 2 4 8....2^{k-1}] 
    #gg=vector([2**i for i in range(k)]) #gadget_vector(k)
    #print("gg=",gg)
    
    # Initialize GG as a zero matrix in ZZ^{n x m}
    GG=zero_matrix(ZZ,n, m)
    #print("GG=")
    #print(GG)
    
    # For each row i, we change elements at i*k+j for j in [0,..,k-1] by gg[j]
    for i in range(n):
        for j in range(k):
            GG[i,i*k+j]=vector([2**t for t in range(k)])[j]# gg[j]
    #print("GG=")
    #print(GG)
    # Change ring of GG into ZZ_q
    GG.change_ring(ZZq)
    
    return GG

#======================================
def random_invertible_matrix(modulus=1, size=3):
    """
    Input:
    - modulus: modulus of the base ring, such as modulus=5 then we consider the ring ZZ_5.
      By default, if modulus=1, we are working in ZZ
    - size: default value is 3
    Output: A matrix that is invertible in ZZ_q, with q=modulus
    """
    # By default, if modulus=1, we are working in ZZ
    if modulus==1:
        V = ZZ**size
        vectors = []
        for i in range(size):
            vv = V.random_element()
            while vv in V.span(vectors):
                vv = V.random_element()
            vectors.append(vv)
    # If modulus q>=2, we are working in ZZ_q        
    if modulus >=2:
        ZZq = IntegerModRing(modulus)
        V = ZZq**size
        vectors = []
        for i in range(size):
            vv = V.random_element()
            while vv in V.span(vectors):
                vv = V.random_element()
            vectors.append(vv)
    return Matrix(vectors)

#======================================================

def perpendicular_q_ary_basis(AA, q):
    """
    *Input:
        - AA: A (n x m)-matrix in ZZq
        - q: a modulus
    *Output: 
        - A basis CC for the q-ary lattice Lambda_q^{perp}(AA) generated by the matrix AA in modulo q, 
        i.e., the lattice L(AA)={x^m in ZZ^m: xx.AA=0 mod q}
        
    """
    # number of rows in AA
    n=AA.nrows()
    m=AA.ncols()
    
    #print("m=",m)
    #print("n=",n)
    
    #Compute “left kernel” (or left_nullspace, i.e. the space of vectors ww such that ww.AA=0
    #left_kernel_of_AA=kernel(AA.transpose()) #or 
    #left_kernel_of_AA=kernel(AA.transpose()) 
    # Note that, we have to perform on input AA.transpose() instead of AA
    WW=Matrix(kernel(AA.transpose()) .basis()).change_ring(ZZ)
    #print("WW=")
    #print(WW)
    del AA
    # Generate matrix q*IIn where IIn is the identity matrix of dimension n
    #qIIn=q*identity_matrix(ZZ,n) 
    #print("q_IIn=")
    #print(q_IIn)
    
    # Generate (n x (m-n))-zero matrix KK
    KK=zero_matrix(ZZ,n, m-n)
    #print("KK=")
    #print(KK)
    
    # Concatenate KK with qIIn
    #BB=KK.augment(q*identity_matrix(ZZ,n)).change_ring(ZZ) # old code
    KK=KK.augment(q*identity_matrix(ZZ,n)).change_ring(ZZ) # new code
    #print("BB=")
   # print(BB)
    
    # The basis is:
    WW=WW.stack(KK)  
    #print("CC=")
    #print(CC)
    
    del KK
    #print("gc.collect()=", gc.collect())
    
    return WW


#=========================
def smoothing_parameter_of_ZZ(epsilon):
    """
    This algorithm computes the smoothing parameter eta_epsilon(ZZ).
    * Input:
        - epsilon: a very small positive real number. E.g., we can set epsilon=2^(-71) or epsilon=2^(-84)
    * Output:
        - smoothing parameter eta_epsilon(ZZ), which is denoted by smt_para
    
    """
    #smt_para=RR(sqrt(ln(2/epsilon)/pi))
    #print("smt_para=", smt_para)
    return RR(sqrt(ln(2/epsilon)/pi))


#======================================================
def smoothing_parameter_q_ary_lattice(AA, q, epsilon):
    """
    This algorithm computes the smoothing parameter eta_epsilon(Lambda_q^{perp}(AA)) for the q-ary lattice Lambda_q^{perp}(AA).
    * Input:
        - AA; a matrix w.r.t the q-ary lattice Lambda_q^{perp}(AA)
        - q: modulus 
        - epsilon: a very small positive real number. E.g., we can set epsilon=2^(-71) or epsilon=2^(-84)
    * Output:
        - smoothing parameter eta_epsilon(Lambda_q^{perp}(AA)), which is denoted by smt_para
    """
    # Compute the basis for the q-ary lattice Lambda_q^{perp}(AA):
    BB=perpendicular_q_ary_basis(AA, q)
    # Dimension of BB (which is a square matrix)
    n=BB.nrows()
    #print("n=",n)
    
    #Compute Gram Schmidt of BB
    BB_gs=BB.gram_schmidt()[0]
    #print("||BB_gs||=", BB_gs.norm())
    #print("factor=", RR(sqrt(ln(2*n*(1+1/epsilon))/pi)))
    
    del BB
    # Compute smoothing parameter of Lambda_q^{perp}(AA) using [MP12,Lemma 2.3] (also [GPV08, Theorem 3.1]) 
    #smt_para=RR(BB_gs.norm()*sqrt(ln(2*n*(1+1/epsilon))/pi))   
    #print("smt_para=",smt_para)
    return RR(BB_gs.norm()*sqrt(ln(2*n*(1+1/epsilon))/pi))


#======================================================
def gen_trap_mp12(n, q, mt, option_HH, epsilon):
    """
    * Input:
        - n: number of rows (security parameter) of AA
        - q: modulus, q should be prime 
        - mt: a positive integer mt>=1
          Then number of columns m=mt+n*k, where k=ceil(log_2(q)), 
        - option_HH: 
                + If option_HH=0 (default) then HH is zero matrix. 
                + If option_HH=1 then HH=II, identity matrix
                + Otherwise, HH is chosen to be invertible in ZZq.   
        - epsilon: epsilon in the smoothing parameter eta_{epsilon}(ZZ) for ZZ.
          We can choose epsilon=2^(-71) or epsilon=2^(-84)
    * Output: 
        - A matrix AA and its trapdoor TT via MP12:
    """
    # Initial
    ZZq = IntegerModRing(q)
    k=ceil(log(q,2))
    w=n*k
    m=mt+w
    #print("mt=", mt)
    #print("nk=", n*k)
    
    #Generate (n x mt)-matrix AAt from ZZq
    AAt = random_matrix(ZZq,n,mt)
    #print("AAt=")
    #print(AAt)
    
    #Generate gadget matrix GG
    GG=gadget_matrix(n, q)
    #print("GG=")
    #print(GG)
    
    #Generate center for Discrete Gaussian
    cc=zero_vector(ZZ,w)
    #print("cc=")
    #print(cc)
    
    
    # Choose Gaussian parameter s for Discrete Gaussian w.r.t. smoothing parameter
    # Here we choose s= smoothing parameter of ZZ (i.e., s=\eta_{epsilon}(ZZ))
    # (See GPV08, at the beginning of Section 2)
    s=smoothing_parameter_of_ZZ(epsilon)
    #print("s=", s)
    
    #sigma= ceil(sqrt(n))#sqrt(log(2+2:e,e):pi)
    #print("sigma=",sigma)
    
    #Generate trapdoor via D_{ZZ**w,s,cc}
    TT=Matrix([sample_d(ZZ**w,s,cc) for i in range(mt)])
    #print("TT=")
    #print(TT)
    
    #Generate n-dimensional matrix H that is invertible in ZZ_q
    if option_HH==0:
        HH=zero_matrix(ZZq,n)
    elif option_HH==1:
        HH=identity_matrix(ZZq,n)
    else:
        HH=random_invertible_matrix(q,n)
    #print("HH=")
    #print(HH)
    
    # Change into ZZ
    HHz=HH.change_ring(ZZ)
    GGz=GG.change_ring(ZZ)
    AAtz=AAt.change_ring(ZZ)
    
    # Compute BBz=HHz*GGz-AAtz*TT in ZZ and take modulo q
    BBz=(HHz*GGz-AAtz*TT)%q
    #print("BBz=")
    #print(BBz)
    del GG
    del GGz
    del AAtz
    del HHz
    
    #print("gc.collect()=", gc.collect())
    #BBz=BBz%q
    
    # Change into ZZq
    BB=BBz.change_ring(ZZq)        
    #print("BB=")
    #print(BB)
    
    # Set AA=[AAt|BB]
    AA=AAt.augment(BB)
    #print("AA=")
    #print(AA)
    
    del AAt
    del BB
    del BBz
    #print("gc.collect()=", gc.collect())
    
    return (AA, HH, TT, q)


#=======================================================

def modulo_inverse_matrix(AA, q):
    """
    * Input: 
        - AA: A matrix
        - q: modulus
    * Output:
        - The inverse matrix modulo q of matrix AA if modulus=q and matrix = AA 
    """
    #AA=matrix
    #q=modulus
    
     # determinant of the matrix AA, i.e., det(AA)
    #determinant=AA.det()   
    #print("determinant=", determinant)
    
    # inverse of the matrix AA
    #inverse_of_AA=AA.inverse() 
    #print("inverse_of_AA=")
    #print(inverse_of_AA)
    
    # inverse modulo q of det(AA) 
    #det_inverse=inverse_mod(q, determinant)                    
    #print("det_inverse=", det_inverse)
    
     # inverse modulo q of matrix AA
    #inverse_of_AA_in_modulo_q=(inverse_mod(q, determinant)*determinant*AA.inverse())%q       
    #print("inverse_of_AA_in_modulo_q=")
    #print(inverse_of_AA_in_modulo_q)
    
    return (inverse_mod(q, AA.det())*AA.det()*AA.inverse())%q     

#=======================================================
def operator_norm(matrix):
    """
    * Input:
        - a matrix, say, LL
        -
    * Output:Compute the operator norm (or sup norm) of matrix, say AA, defined as s_1(LL)=sup(||LLuu||/||uu||).
    * Steps:
        - Trasform the base-ring of the matrix into the RDF ring
        - Perform the Singular Value Decomposition to get LL=AA.BB.CC in which BB is a diagonal matrix and s_1(LL)=BB[0][0]. 
    """
    matrix=matrix.change_ring(RDF)
    #print("matrix=")
    #print(matrix)
    
    # Singular Value Decomposition of 
    (AA,BB,CC)=matrix.SVD()
    #print("AA=")
    #print(AA)
    #print("BB=")
    #print(BB)
    #print("CC=")
    #print(CC)
    del AA
    del CC
    return BB[0][0]


#=======================================================

def basis_of_gadget_vector(q):
    """
    This algorithm is to generate the basis for the q-ary lattice Lambda_q^{perp}(gg),
    where gg=[1 2 4 ... 2^{k-1}] is the gadget vector. 
    
    * Input:
        - q: modulus
    * Output:
        - basis SS_k for the q-ary lattice Lambda_q^{perp}(gg) w.r.t 
          the gadget vector gg=[1 2 4 ... 2^{k-1}], with k=ceil(log_2(q))
    
    """
 
    # bit size of q
    k=ceil(log(q,2))
    
    # We compute the bit decomposition of q below.
    
    # However, this way is not correct as it will return something like this:
    # bits_of_q_1= ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
    # ===> BE CAREFUL!! 
    # q_bits_1=bin(q)[2:]
    # print("q_bits_1==bin(q)[2:]=",q_bits_1)
    # bits_of_q_1=[q_bits_1[i] for i in range(len(q_bits_1))]#q.bits()
    # print("bits_of_q_1=", bits_of_q_1)
    
    # We have to use the following way:
    bits_of_q=q.bits()
    #print("bits_of_q=", bits_of_q)
    
    # Initialize SS_k as a zero matrix in ZZ^{k x k}
    SSk=zero_matrix(ZZ, k, k)
    # print("SS_k=")
    # print(SSk)
    
    # For each column i
    for j in range(k-1):
        SSk[j,j]=2
        SSk[j+1,j]=-1
        
    # For the last column, i.e., j=k-1 
    # if q=2^k, we set the (k,k)-th element to be 2
    if q==2**k:
            SSk[k-1,k-1]=2
    # if q!=2^k , we change the last column by the list bits_of_q
    else:
        for i in range(k):
            SSk[i,k-1]=bits_of_q[i]
    # print("SS_k=")
    # print(SSk)
    
    return SSk
    
    

    
    
#=======================================================
def basis_of_gadget_matrix(n,q):
    """
    This algorithm is to generate the basis for the q-ary lattice Lambda_q^{perp}(GG), where
                           |gg  0   0  ...   0 |
                           |0   gg  0  ...   0 |
                      GG=  |0   0   gg ...   0 |    
                           |...................|
                           |0   0    0 ...   gg|,
     where gg=[1 2 4 ... 2^{k-1}] is the gadget vector. 
     
    * Input:
        - n: number of rows of the gadget matrix GG
        - q: modulus
    * Output:
        - basis SS for the q-ary lattice Lambda_q^{perp}(GG).
    """
     
    # Conpute a basis SSk for the q-ary lattice Lambda_q^{perp}(gg), 
    # where the gadget vector gg=[1 2 4 ... 2^{k-1}]
    #SSk=basis_of_gadget_vector(q)
    #IIn=identity_matrix(ZZ,n)
    
    # SS is the tensor product of IIn with SSk
    #SS=IIn.tensor_product(SSk)
    return identity_matrix(ZZ,n).tensor_product(basis_of_gadget_vector(q))  #SS



#=======================================================

#======= This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k ========
def sample_from_gadget_vector_modulus_power_of_2(q, u):
    """
   This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k.
   It implements the second approach of Gaussian sampling in [MP12, Section 4.1] for the case q=2^k
    * Input:
        - q: modulus which is q=2^k, where k is positive integer. 
             Remember that k is the dimension of the gaget vector gg=(1, 2,..., 2^{k-1})
        - u: a coset, which is an integer in {0,...,q-1}

    * Output: a list xx in ZZ^k s.t. vec(xx) satisfying that <gg,vec(xx)>=u mod q, 
              where gg=(1 2 4 ... 2^{k-1}) with probability proportional to Gaussian rho_s(xx),
              and s is Gaussian parameter.
    """
    #Gausian parameter which is an optimal bound on the smoothing parameter of \Lambda^{perp}(GG) 
    epsilon=2**(-90)
    sigma=2*smoothing_parameter_of_ZZ(epsilon)
    k=log(q,2)
    #
    list_of_vector_x=[]
    
    # First we sample z_i form D_{ZZ,s/2,-u/2} and then we compute x_i=2z_i+u. 
    # Then, we assign -z_i to u.
    for i in range(k):
        z_i=sample_z(sigma/2,-ceil(u/2))
        x_i=2*z_i+u
        u=-z_i
        list_of_vector_x.append(x_i)
        
    #xx = vector(list_of_vector_x)
    #return vector(list_of_vector_x)    
    return list_of_vector_x

#=======================================================
#======= This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k ========
def sample_from_gadget_matrix_modulus_power_of_2(q, uu, n):
    """
    This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k.
    It is used to sample vectors xx such that GG*xx=uu mod q via Gaussian.
    It implements the second approach of Gaussian sampling in 
    [MP12, Section 4.1] for the case q=2^k by calling 
    sample_from_gadget_vector_modulus_power_of_2(q, u) n times
    * Input:
        - q: modulus which is q=2^k, k is positive integer. 
             Remember that the dimension k of gaget vector gg=(1, 2,..., 2^{k-1})
        - u: a coset, which is an integer in {0,...,q-1}
        - n: number of rows in         
                                         |gg  0   0  ...   0 |
                                         |0   gg  0  ...   0 |
                      Gadget matrix GG=  |0   0   gg ...   0 |    
                                         |...................|
                                         |0   0    0 ...   gg|
    * Output: a vector xx in ZZ^k s.t. GG*xx=uu mod q, where gg=(1 2 4 ... 2^{k-1})
    """
    xx_list=[]
    for i in range(n):
        xx_i = sample_from_gadget_vector_modulus_power_of_2(q, uu[i])
        i+=1
        #print("xx_i=",xx_i)
        xx_list=xx_list+xx_i
    return xx_list
    #print("xx_list=",xx_list)
#=======================================================


#======= This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k ========

def sample_from_gadget_matrix_modulus_power_of_2_generator_version(q, uu, n):
    """
    This algorithm is excellent in speed compared with sample_from_gadget_matrix_modulus_power_of_2.
    It exploits the generator mechanisnm in Python.
    
    This algorithm is used when q IS EXACTLY a power of 2, i.e, q=2^k.
    It is used to sample vectors xx such that GG*xx=uu mod q via Gaussian.
    It implements the second approach of Gaussian sampling in 
    [MP12, Section 4.1] for the case q=2^k by calling 
    sample_from_gadget_vector_modulus_power_of_2(q, u) n times
    * Input:
        - q: modulus which is q=2^k, k is positive integer. 
             Remember that the dimension k of gaget vector gg=(1, 2,..., 2^{k-1})
        - u: a coset, which is an integer in {0,...,q-1}
        - n: number of rows in         
                                         |gg  0   0  ...   0 |
                                         |0   gg  0  ...   0 |
                      Gadget matrix GG=  |0   0   gg ...   0 |    
                                         |...................|
                                         |0   0    0 ...   gg|
    * Output: a vector xx in ZZ^k s.t. GG*xx=uu mod q, where gg=(1 2 4 ... 2^{k-1})
    """
    epsilon=2**(-90)
    sigma=2*smoothing_parameter_of_ZZ(epsilon)
    
    #print("sigma=", sigma)
    k=log(q,2)
    for j in range(n):
        for i in range(k):
            z_i=sample_z(sigma/2,-ceil(uu[j]/2))
            xx_i=2*z_i+uu[j]
            uu[j]=-z_i
            yield xx_i
            i+=1
        j+=1
        
#=======================================================
def sample_d_mp12(TT,AA,HH,uu, q, epsilon=2^(-84)):
    """
    This algorithm is to sample vector from discrete Gaussian over q-ary lattice Lambda^{uu}^{perp}(AA);
    * Input:
        - TT: An (mt x w)-matrix trapdoor for matrix AA (TT is exactly RR in [MP12]) in ZZq.
          Here w=n*k, where k=ceil(log_2(q))
        - AA: A random (n x m)-matrix in ZZq. 
          Note that AA is not AAbar in Algorihtm 3 of MP12.
        - HH: An invertible (n x n)-matrix in ZZ_q (it is called the tag w.r.t AA, TT).
          However, we compute inverse of HH in ZZ instead of in ZZ_q. 
          If HH=zero_matrix then the algorithm will be stopped.
        - uu: An n-vector in ZZq
        - q: A modulus for the ring ZZq we are working in
        - epsilon: For smoothing parameter. By default, we set epsilon=2^(-84)
    * Output: 
        - A (pseudo-random) vector xx in the q-ary lattice Lambda_{uu}^{perp}(AA) 
          sampled from D_{Lambda_{uu^{perp}(AA)}
    """
    # Check if HH is invertible
    #if HH.is_invertible()==False:
        #print("The algorithm does not work in this case. Please choose the tag matrix to be invertible.")
        #return ValueError
    # Dimensions, sizes 
    n=AA.nrows()
    #print("n=", n)
    m=AA.ncols()
    mt=TT.nrows()   
    #print("mt=", mt)
    w=TT.ncols()         # Note that, m=mt+w
    #print("w=", w)
    
    k=ceil(log(q,2))
    
    
    ZZq = IntegerModRing(q)
 
    # operator norm of TT
    s1_TT=operator_norm(TT)
    
    # Gaussian parameters
    r=smoothing_parameter_of_ZZ(epsilon) # Here we choose r= smoothing parameter of ZZ (i.e., s=\eta_{epsilon}(ZZ))
                                         # (See GPV08, at the beginning of Section 2)
    sigma_GG = 5                 # sigma_GG: Gaussian parameter used to sample over the q-ary 
                                 # lattice Lambda_q^{perp}(GG). 
                                 # Via [MP12, Algorithm 3], sqrt(sigma_GG)>=2 or sigma_GG=sqrt(5). 
    sigma = 7*(s1_TT**2+1)
    #sqrt_sigma=sqrt(sigma)
    sigma_pp = 2*(s1_TT**2+1)
    r_GG = r*sqrt(sigma_GG)
    r_pp = r*sqrt(sigma_pp)
  
    # Generate gadget matrix GG
    #GG = gadget_matrix(n, q)
    #GGz=GG.change_ring(ZZ)
    #print("GG=")
    #print(GG)
    #print("GG.nrows=", GG.nrows())
    #print("GG.ncols=", GG.ncols())
    
    # Change ring for uu
    #uuz=uu.change_ring(ZZ)  # We have to change ring. Otherwise, it will get an error notification.
    #print("uuz=",uuz)
     
    
    #Compute [TT^t I] 
    IIw = identity_matrix(ZZ,w)
    KK = (TT.transpose()).augment(IIw)
    #print("KK=")
    #print(KK)
        
    # Stack of TT over IIw, i.e., matrix [TT^t|IIw]^t
    TT_over_IIw=TT.stack(IIw)
    
    # Choose pp from D_{ZZ^m,r_pp}====
    cc=zero_vector(ZZ,m)        # center vector
    pp=sample_d(ZZ**m,r_pp,cc)
    del cc
    #print("gc.collect()=", gc.collect())
    # print("pp=", pp)
    pp1=vector([pp[i] for i in range(mt)])
    pp2=vector([pp[i] for i in range(mt, m)])
    # print("pp1=", pp1)
    # print("pp2", pp2)
    
    #print("len(pp)=",len(pp))
    #print("len(pp1)=",len(pp1))
    #print("len(pp2)=",len(pp2))
    # Take matrix AA bar from matrix AA
    #AAt=AA[0:n,0:mt] # Submatrix of AA consisting of rows 0...n-1, cols 0...mt-1  # old code
    #AAt= AAt.change_ring(ZZ)
    
    AA=AA[0:n,0:mt] # Submatrix of AA consisting of rows 0...n-1, cols 0...mt-1
    AA= AA.change_ring(ZZ)
    # print("AAt=");print(AAt)
   
    # vector ww bar in MP12
    wwt=(AA*(pp1-TT*pp2))%q
    ww=(gadget_matrix(n, q)*pp2)%q
    del AA
    del pp1
    del pp2
    del TT
    del IIw
    #print("gc.collect(), AA, pp1, pp2, TT, IIw=", gc.collect())
    
    # wwtz=wwt.change_ring(ZZ)
    # wwz=ww.change_ring(ZZ)
    #print("wwtz=",wwt)
    #print("wwz=",ww)
    
    #Compute inverse of the tag HH
    #HH_inv=HH.inverse() # If we generate HH_inv <----modulo_inverse_matrix(HH, q) it will be false
    #print("HH_inv=")
    #print (HH_inv)
    #HH_invz=(HH_inv).change_ring(ZZ)   # Old code
    
    
    
    #---- NOTE-------
    # We consider the case HH=II then HH.inverse()=HH then we do not compute 
    # the following to save time and save memory
    #HH=(HH.inverse()).change_ring(ZZ)
    # In general, we have to compute HH.inverse()
    #---- NOTE ------
    
    
        
    #t1=timer()
    vv=(HH.change_ring(ZZ)*(uu.change_ring(ZZ)-wwt)-ww)%q
    del uu
    del wwt
    del ww
    del HH
    #print("gc.collect()=", gc.collect())
    #t2=timer()
    #print("time to compute vv=", t2-t1)
    #print("len(vv)=",len(vv))
    # vv=vv.change_ring(ZZq)
    #print("vv=",vv)
    
    # Find a vector tt in ZZ^{w} such that GGz*tt=vvz. 
    # We can easily see that tt=bit_decomposition of vvz, computed as follows:
    # (i) each element of vvz (which is in ZZq) will have at most k=ceil(log(q,2)) bits.
    # (ii) if any element does not have enough k bits, then we append zeros to get k bits. 
    
    # ===== NO NEED ANYMORE ====================
    #t1=timer()
    #bit_decomp_of_vv=[]
 
    #for i in range(len(vv)):
        #list_of_bits=vv[i].bits()
        #while len(list_of_bits)<k:
            #list_of_bits.append(0)
        # print("vv[i].bits()=",list_of_bits)
        #for j in range(k):
            #bit_decomp_of_vv.append(list_of_bits[j])
    # print("bit_decomp_of_vv=",bit_decomp_of_vv) 
    #tt=vector(bit_decomp_of_vv)
    #t2=timer()
    #print("time for bit_decomp_of_vv=", t2-t1)
    
    #===============================================
    #print("tt=",tt)
    #print("len(tt)=",len(tt))
    # print(vv==GG*tt) #to check if GG*tt=vv
    
    # Basis SS for q-ary lattice w.r.t GG
    #t1=timer()
    #SS=basis_of_gadget_matrix(n,q)
    #t2=timer()
    #print("time for basis_of_gadget_matrix=", t2-t1)
    #print("SS=")
    #print(SS)
    #print("SS.nrows()=",SS.nrows())
    #print("SS.ncols()=",SS.ncols())
    
    # EE=GGz*SS%q
    # print("EE=")
    # print(EE)
  
    #=============== NO NEED ANYMORE ( VER SLOW) =====================
    # Choose zz from Lamda_vv^{\perp}(GG) using Discrete Gaussian of parameter r*sqrt(sigma_GG)
    #sample_d(SS.transpose(), r_GG, -tt) 
    #===============================================================
    
    #==THIS WAY IS SO GOOD =======
    #print("compute zz")
    #t1=timer()
    zz= vector(ZZ, sample_from_gadget_matrix_modulus_power_of_2_generator_version(q, vv, n))
    # Note that, we input SS.transpose() instead of SS
    #t2=timer()
    #print("      time for xx in sample_from_gadget_matrix_modulus_power_of_2_generator_version(q, vv, n) in sample_d_mp12", t2-t1)
    del vv
    #print("gc.collect() in sample_d_mp12=", gc.collect())
    
    #print("time for zz by sample_from_gadget_matrix_modulus_power_of_2_generator_version(q, vv, n)=", t2-t1)
    #print("yy=")
    #print(yy)
    
    #print("GG*yy=", (GG*yy)%q)
    
    #zz=yy+tt
    #print("zz=")
    #print(zz)
    
    # Lattice vector that is sampled from D_{Lambda_{uu^{perp}(AA)} will be:
    #t1=timer()
    xx= pp+TT_over_IIw*zz
    #t2=timer()
    #print("time for xx in sample_d_mp12=", t2-t1)
    #assert (AA*xx)%q==uu
    #print("AA*xx=", (AA*xx)%q)
    #print(AA*xx==uu)
    del TT_over_IIw
    del zz
    del pp
    #print("gc.collect() deleting TT_ove_IIw, zz, pp=", gc.collect())
    
    return xx


#=======================================================


def del_trap_mp12(AA, HH, TT, AA_1, HH_new, q, epsilon=2^(-90)):
    """
    See Algorithm 4 in MP12.
    * Input:
        - AA: A (n x m)-dimensional matrix 
        - AA_1: A (n x n*k)-dimensional matrix
        - HH: tag matrix for AA
        - TT: GG-trapdoor for AA
        - HH_new: An invertible matrix of dimension (m+n*k)x(m+n*k)
        - q: A modulus 
        - epsilon: To control the smoothing parameter
          By defaul we choose 2^(-84)
    * Output:
        - Output the MP12 trapdoor TT_new for AA_new=[AA|AA_1]
    """
    # Check if HH is invertible
    #if HH_new.is_invertible()==False:
        #print("The matrix is not invertible. Please choose the tag matrix to be invertible.")
        #return ValueError
    
    # Dimensions, sizes 
    n=AA.nrows()
    #print("n=", n)
    m=AA.ncols()   
    #print("m=", m)
    mt=TT.nrows()   
    #print("mt=", mt)
    w=TT.ncols()         # Note that, m=mt+w
    #print("w=", w)   
    k=ceil(log(q,2))
    
    #print("k=",k)
    
    #print("AA.nrows()=",AA.nrows())
    #print("AA.ncols()=",AA.ncols())
    
    #print("TT.nrows()=",TT.nrows())
    #print("TT.ncols()=",TT.ncols())
    
    #print("AA_1.nrows()=",AA_1.nrows())
    #print("AA_1.ncols()=",AA_1.ncols())
    
    # Generate gadget matrix GG
    #t1=timer()
    GG=gadget_matrix(n, q)
    #t2=timer()
    #print("time for GG=", t2-t1)
    
    # Gaussian parameter:
    # We do not need this one, since via sample_d_mp12, the Gaussian parameter only depends on
    # the operator norm s_1(TT) which can be computed inside sample_d_mp12
    #s_new=smoothing_parameter_q_ary_lattice(AA,q,epsilon)
    #print("s_new=",s_new)
    
    #print("AA=")
    #print(AA)
    
    # Compute matrix UU=HH_new*GG-AA_1
    #UU=HH_new*GG-AA_1                                # (old code)
    #UU_in_column_form=UU.transpose()
    #t1=timer()
    
    UU_in_column_form=(HH_new*GG-AA_1).transpose()    # (new code)
    
    del GG
    del AA_1
    del HH_new
    #print("gc.collect()=", gc.collect())
    
    
    #t2=timer()
    #print("time for UU_in_column_form=", t2-t1)
    #print("UU=")
    #print(UU)
    #print("UU_in_column_form=")
    #print(UU_in_column_form)
    #print("UU.nrows()=",UU.nrows())
    #print("UU.ncols()=",UU.ncols())
    
    # Using sample_d_mp12 to generate tt_new for each column of UU
    #TT_new_transpose=zero_matrix(ZZ,w, m)  # old code
    TT_new=zero_matrix(ZZ,w, m)  # new code
    
    #print("TT_new_transpose=")
    #print(TT_new_transpose)
 
    #list_to_matrix=[]
    print("********************")
    print("del_trap_mp12 has to call sample_d_mp12 ", w, "times:")
    for i in range(w):
        
        print("    The ",i+1, "-th call:")
        #tt_new=sample_d_mp12(TT,AA,HH,UU_in_column_form[i],q, epsilon)
        #print("tt_new=")
        #print(tt_new)
        #print(AA*tt_new==UU_in_column_form[i])
        #TT_new[i]=sample_d_mp12(TT,AA,HH,UU_in_column_form[i],q, epsilon)  # old code
        t1=timer()
        TT_new[i]=vector(ZZ,sample_d_mp12(TT,AA,HH,UU_in_column_form[i],q, epsilon))  # new code
        t2=timer()
        print("                       Time=", t2-t1)
        #print(AA*tt_new==UU_in_column_form[i]%q)
        #list_to_matrix.append(tt_new)
    del UU_in_column_form
    del TT
    del AA
    del HH
    print("********************")
    #print("gc.collect()=", gc.collect())
    
    
    #TT_new=(Matrix(list_to_matrix)).transpose() # It is a column matrix
    
    #TT_new=TT_new.transpose()
    #print("TT_new=")
    #print(TT_new.str())
    
    # To check
    #print("AA*TT_new=UU?",AA*TT_new==UU)
    #print("TT_new.nrows()=",TT_new.nrows())
    #print("TT_new.ncols()=",TT_new.ncols())
    
    return TT_new.transpose()

#=================================================================


###========================================================###
###========================================================###
###         IDENTITY-BASED LINKABLE RING SIGNATURE         ###
###                   (HASH FUNCTION)                      ###
###========================================================###
###========================================================###




def hash_to_a_matrix(string,n,m,q):
    """
    Hash function h: {0,1}^* |---->  ZZ_q^{n x m} 
    that maps a bit string of abitrary length to a random matrix in ZZ_q^{n x m}.
    The idea is as follows:
    -  On input "string", we use SHAKE-256 to generate a binary string bin_str of M=n*m*ceil(log_2(q)) bits.
     Note that, in hexdigest(par), we choose par=ceil (M/8).
    -  We use ceil(log_2(q)) bits to convert an integer in ZZ_q
    * Input:
        - string: A string
        - q: modulus
        - n, m: Integers define the dimesions of output matrices.
    * Output:
        - a random matrix in ZZ_q^{n x m}
    """
    # 
    ZZq = IntegerModRing(q)
    k = ceil(log(q,2))
    
    AA = Matrix(ZZq,n, m)
    s = hashlib.shake_256()
    s.update(string.encode())
    
    # Compute the input for hexdigest(). With input t, it outputs about t*8 bits.
    # We will use k (here k=ceil(log(q,2))) bits to get an integer in ZZ_q, 
    # then we need n*m*k bits for ZZ_q^{n x m}
    t=ceil(n*m*k/8)
    #print("t=",t)
    hex_string=s.hexdigest(int(t+1)) # We put t+1 to make sure the sufficient number of output bits  
    a=Integer(hex_string, base=16)      
    
    # binary string of the integer a
    a_bits=bin(a)[2:]     # We don't care the prefix "0b" 
    #print("len(a_bits)=", len(a_bits))
    #print("a_bits=",a_bits)
    
    for i in range(n):
        #cut a_bits into n intervals, each of which having size m
        str_i=a_bits[i*m*k:(i+1)*m*k]
        #print("len(str_", i, ")=", len(str_i))
        #print("str[",i,"j=", str_i)
        for j in range(m):
            #cut str_i into n*k intervals, each of which having size of k=ceil(log_2(q))
            aa_ij=str_i[j*k:(j+1)*k]
            #print("aa_ij=",aa_ij)
            
            #Convert into the (i,j)-the element of AA
            AA[i,j]=int(aa_ij,base=2)
            #print("AA[",i, ",",j,"]=",AA[i,j])
            
    #print("AA=")
    #print(AA)
    
    return AA

#===============================================
def hash_to_a_matrix_generator_version(string,n,m,q):
    """
    This algorithm is as same as hash_to_a_matrix(string,n,m,q), 
    except that it uses the generator mechanism in Python.
    
    Hash function h: {0,1}^* |---->  ZZ_q^{n x m} 
    that maps a bit string of abitrary length to a random matrix in ZZ_q^{n x m}.
    The idea is as follows:
    -  On input "string", we use SHAKE-256 to generate a binary string bin_str of M=n*m*ceil(log_2(q)) bits.
     Note that, in hexdigest(par), we choose par=ceil (M/8).
    -  We use ceil(log_2(q)) bits to convert an integer in ZZ_q
    * Input:
        - string: A string
        - q: modulus
        - n, m: Integers define the dimesions of output matrices.
    * Output:
        - a random matrix in ZZ_q^{n x m}
    """
    # 
    ZZq=IntegerModRing(q)
    k=ceil(log(q,2))
    
    s = hashlib.shake_256()
    s.update(string.encode())
    
    # Compute the input for hexdigest(). With input t, it outputs about t*8 bits.
    # We will use k (here k=ceil(log(q,2))) bits to get an integer in ZZ_q, 
    #then we need n*m*k bits for ZZ_q^{n x m}
    t = ceil(n*m*k/8)
    #print("t=",t)
    hex_string=s.hexdigest(int(t+1)) # We put t+1 to make sure the sufficient number of output bits  
    a = Integer(hex_string, base=16)      
    
    # binary string of the integer a
    a_bits = bin(a)[2:]     # We don't care the prefix "0b" 
    #print("len(a_bits)=", len(a_bits))
    #print("a_bits=",a_bits)
    
    for i in range(n):
        #cut a_bits into n intervals, each of which having size m
        str_i = a_bits[i*m*k:(i+1)*m*k]
        #print("len(str_", i, ")=", len(str_i))
        #print("str[",i,"j=", str_i)
        aa = [0]*m
        for j in range(m):
            
            #cut str_i into n*k intervals, each of which having size of k=ceil(log_2(q))
            aa_ij = str_i[j*k:(j+1)*k]
            #print("aa_ij=",aa_ij)
            
            #Convert into the (i,j)-the element of AA
            aa[j] = int(aa_ij,base=2)
        yield aa
        i+=1
   

#====================
def hash_to_a_ball(string,n,w):
    """
    This is the hash function h_2: {0,1}^* |---->  S^n_w
    that maps a bit string of abitrary length to a random sparse vector in S^n_w.
    Here S^n_w:{cc in {0,1}^n: ||cc||_1=w}.
    
    The idea is as follows:
    - Exploit the idea of SampleInBall of Dilithium 
    (see at https://pq-crystals.org/dilithium/index.shtml)
    - 
    * Input:
        - string: A string
        - n: length of output binary vectors
        - w: the number of bit 1 in the output vector.
    
    * Output:
        - a random vector in S^n_w
    """
    # Hash the input string using SHAKE-256 to get a 256-bit output
    a_t = hashlib.shake_256()
    a_t.update(string.encode())
    a=Integer(a_t.hexdigest(int(w+10)), base=16)      # w+1 bytes to have an integer of (w+10)*8 bits 
    
    # binary string of the integer a
    a_bits=bin(a)
    #print("len(a_bits)=", len(a_bits))
    #print("a_bits=",a_bits)
    
    # From now on, we impplement SampleinBall of Dilithium
    cc=[0 for i in range(n)]
    #print("cc=",cc)
    
    pos=7 # We start by using the first 8 bits of a
    count=0
    
    for i in range(n-w,n):
        #print("i=",i)
        j=i+1           # set j >i
        while (j>i):
            #count=count+1
            
            # To check the length of the last block of bits in the current bit string.
            # If the length <8, we have to call SHAKE-256 again to get a longer bit string
            
            #print("len(a_bits[pos+1:pos+9])=", len(a_bits[pos+1:pos+9]))
            if(len(a_bits[pos+1:pos+9])<8 ):
                #print("Create a new bit string with count=", count)
                a=Integer(a_t.hexdigest(int(w+10+count)), base=16)          # (w+10+count) bytes
                a_bits=bin(a)
                #print("len(a_bits)=", len(a_bits))
                #print("a_bits=",a_bits)
                #pos=255+(count-1)*8
                
            # Compute new j that belongs to {1, ..., i}
            #==========================================
            
            # 8 bits in [pos+1, pos+9)
            j_bits=a_bits[pos+1:pos+9]   
            #print("j_bits=",j_bits)
            j=Integer(j_bits, base=2)
            
            # Count the muber of repeating the While loop due to j>i
            if (j>i):
                #print("We have (j>i), j=",j)
                count=count+1
                #print("count=", count)
        
            # Move to the next byte
            pos=pos+8
            #print("pos in while loop =",pos)
        
        
        #print("We have (j<=i), j=",j)
        #print("pos out while loop=",pos)
        #print("cc[j]=",cc[j])
        cc[i]=cc[j]
        #print("cc[i]=",cc[i])
        cc[j]=1
        #print("cc_new[j]=",cc[j])
        #print("=======================================")
    
    #print("cc=",cc)
    return vector(cc)












###========================================================###
###========================================================###
###         IDENTITY-BASED LINKABLE RING SIGNATURE         ###
###                        (MAIN PART)                     ###
###========================================================###
###========================================================###



#====================

def idlrs_parameter_setup():
    
    print("Please enter the security_parameter n=: \n")
    n=int(input())
    #print("We have n=secuirty_parameter=\n", n)
    t=log(n,2)
    vp=2+3*t+log(t,2)
   
    #print("Please enter  k (q=2^k) s.t. k-log_2(k)>=",ceil(vp))
    print("Please enter k: \n")
    k=int(input())
    
    
    #print("Please enter a modulus q>=2:\n")
    q=2**k
    print("q=2^",k, "=", q)
    
    print("Please enter mt>=",n*k)
    mt= int(input())
    m=mt+n*k
    print("We have m=mt+n*k=",m)
    
    
    #print("Please enter min-entropy of the hash function H_2, alpha=n")
    #alpha=n
    w=ceil(n/2)
    #while 2^w*binomial(n,w)< 2^alpha:
        #print("Please enter w s.t. 2^w*binomial(n,w)>=", 2^alpha)
        #w= int(input())
    
    
    M=3
    #k=ceil(log(q,2))
    
    
    #print("Please enter epsilon_power (for the smoothing parameter and Gaussian parameters)")
    #print("then we can compute epsilon=2^(-epsilon_power)")
    #print("We should choose epsilon_power>=84:\n")
    #epsilon_power= 90 #int(input())
    epsilon=2**(-90)
    
    # Gaussian parameter for gen_trap_mp12
    sigma_1=smoothing_parameter_of_ZZ(epsilon)
    
    # Gaussian parameter for del_trap_mp12
    s_1_TT=RR(sigma_1*(1/sqrt(2*pi))*(sqrt(mt)+sqrt(n*k)))
    sigma_2=RR(sqrt(5)*(s_1_TT+1)*(sqrt(log(n,2))))
    
    # Gaussian parameter for sample_d_mp12
    s_1_TT_id=RR(sigma_2*(1/sqrt(2*pi))*(sqrt(m)+sqrt(n*k)))
    sigma_3=RR(sqrt(7*(s_1_TT_id^2+1)))
    
     # Gaussian parameter for rejection sampling
    sigma=RR(sigma_3*sqrt(m+n*k)*w*sqrt(log(m+n*k+n,2)))
    
    #assert q >= sqrt(m)*q^(n/m)*sqrt(n*log(n,2))
    #assert q>=2*1.1*sigma*sqrt(m+n*k+n)#>=sqrt(m)*q^(n/m)
    #assert 2^w*binomial(n,w)>=2^alpha
    
    public_parameters=(n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)
    #print("public_parameters=(n,q, w, M, m, mt)=", public_parameters)
    
    # create a txt file containing the main public parameters n, q, mt and w.
    file_params= open("idlrs_parameters.txt", "a")
    file_params_info = str(n) + ',' +  str(q) + ',' + str(mt) + ','+ str(w) + ','+ str(m) + ','+ str(M) +','+ str(sigma_1) + ','+ str(sigma_2) + ','+ str(sigma_3) + ','+ str(sigma) +'\n' # each line
    file_params.write(file_params_info) 
    file_params.close()
     
    
    return public_parameters


#============================================

def idlrs_keygen(public_parameters, option_HH=1):
    """
    This algorithm is to generate public key and secret key for the scheme
    * Input:
        - public_parameters: public parameters that was generated by the algorithm 
          parameter_setup(secuirty_parameter). Note that, public_parameters=(n, q, w, M, m, mt).
        - option_HH: option to generate the tag matrix HH. 
            + If option_HH=0: HH= zero matrix
            + If option_HH=1 (default)): HH is chosen to be the identity matrix II
            + Otherwise: HH will be randomly invertible matrix
    
    * Output:
        - A key pair (public_key, secret_key)
    """
    (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=public_parameters
    #option_HH=1            # HH is chosen to be the identity matrix II
    epsilon=2**(-90)        # for Gaussian paramater
    
    #t1=timer()
    #(AA, HH, TT, q)=gen_trap_mp12(n, q, mt, option_HH, epsilon) # old code
    (master_public_key, tag_matrix, master_secret_key, q)=gen_trap_mp12(n, q, mt, option_HH, epsilon) # new code
    #t2=timer()
    #print("time for gen_trap_mp12=", t2-t1)
    #master_public_key=AA
    #master_secret_key=TT
    #print("master_public_key=")
    #print(master_public_key)
    #print("master_secret_key=")
    #print(master_secret_key)
    #tag_matrix=HH
    #print("tag matrix HH=")
    #print(tag_matrix)
    #keys=(master_public_key, master_secret_key, tag_matrix)
    return (master_public_key, master_secret_key, tag_matrix) #=keys

#=====================================================
def public_key_for_each_identity(public_parameters, master_public_key, identity):
    """
    """
    (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=public_parameters
    q=Integer(q)
    k=ceil(log(q,2))
    
    #print("m+n*k+n=",m+n*k+n)
    #print("#bit of q, k=log_2(q)=",k)
    
    #print("(n, q, w, M, m, mt,k)=",(n, q, w, M, m, mt,k))
    
    ZZq=IntegerModRing(q)
    
    #print(type(q))
    
    #AA = master_public_key
    #print("AA=")
    #print(AA)
    
    #QQ_id=hash_to_a_matrix(identity,n,n*k,q)
    #print("QQ_id=")
    #print(QQ_id)
    #print("QQ_id.nrows()=",QQ_id.nrows())
    #print("QQ_id.cols()=",QQ_id.ncols())
    
    #AA_id=AA.augment(hash_to_a_matrix(identity,n,n*k,q))
    #print("AA_id=")
    #print(AA_id)
    #print("AA_id.nrows()=",AA_id.nrows())
    #print("AA_id.cols()=",AA_id.ncols())
    
    return master_public_key.augment(Matrix(ZZq,hash_to_a_matrix_generator_version(identity,n,n*k,q)))

#===========================


def unit_vector_i(i,n):
    """
    Generate a unit vector having 1 at the i-th element
    """
    xx=zero_vector(ZZ,n)
    xx[i]=1
    return xx

#================================

def idlrs_extract(public_parameters, keys, identity):
    """
    
    """
    
    epsilon=2**(-90) # For smoothing parameter in Gaussian distribution
    
    (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=public_parameters
    q=Integer(q)
    k=ceil(log(q,2))
    #print("#bit of q, k=log_2(q)=",k)
    
    #print("(n, q, w, M, m, mt, sigma_1, sigma_2, sigma_3, sigma)=",(n, q, w, M, m, mt, sigma_1, sigma_2, sigma_3, sigma))
    
    ZZq=IntegerModRing(q)
    
    #print(type(q))
    
    #(master_public_key, master_secret_key, tag_matrix)=keys
    #(AA,TT,HH)=keys
    
    #AA=master_public_key
    #print("AA=")
    #print(AA)
    #TT=master_secret_key
    #print("TT.nrows()=",TT.nrows())
    #print("TT.cols()=",TT.ncols())
    #HH=tag_matrix
    #print("HH=")
    #print(HH)
    #print("HH.nrows()=",HH.nrows())
    #print("HH.cols()=",HH.ncols())
    #t1=timer()
    QQ_id=Matrix(ZZq, hash_to_a_matrix_generator_version(identity,n,n*k,q))
    #t2=timer()
    #print("time for QQ_id=", t2-t1)
    #print("QQ_id=")
    #print(QQ_id)
    #print("QQ_id.nrows()=",QQ_id.nrows())
    #print("QQ_id.cols()=",QQ_id.ncols())
    
    AA_id=keys[0].augment(QQ_id)   #keys[0]=AA
    #print("AA_id=")
    #print(AA_id)
    #print("AA_id.nrows()=",AA_id.nrows())
    #print("AA_id.cols()=",AA_id.ncols())
    
    #random_invertible_matrix(modulus=1, size=3)
    #HH_id=HH=keys[2] # We set H_id to be HH instead of choosing randomly via random_invertible_matrix(q, n).
    #print("HH_id=")
    #print(HH_id)
    #print("HH_id.nrows()=",HH_id.nrows())
    #print("HH_id.cols()=",HH_id.ncols())
    
    #del_trap_mp12(AA, HH, TT, AA_1, HH_id, q, epsilon)
    #TT_id=del_trap_mp12(AA,HH,TT, QQ_id, HH_id, q, epsilon) # old code
    #t1=timer()
    print("del_trap_mp12 starts")
    TT_id=del_trap_mp12(keys[0],keys[2],keys[1], QQ_id, keys[2], q, epsilon)  # new code, AA_1=QQ_id
    #t2=timer()
    #print("time for del_trap_mp12 in idlrs_extract=", t2-t1)
    #print("TT_id.nrows()=",TT_id.nrows())
    #print("TT_id.cols()=",TT_id.ncols())
    print("del_trap_mp12 ends")
    #SS_id_transpose=zero_matrix(ZZ, n, m+n*k)   # old code
    SS_id=zero_matrix(ZZ, n, m+n*k)             # new code
    
    # matrix q*II_n
    #qIIn=q*identity_matrix(ZZq,n)
    
    print("********************")
    print("sample_d_mp12 in idlrs.extract has to run", n, "loops")
    for i in range(n):
        #print("sample_d_mp12 in idlrs.extract has to run", n, "times")
        print("          This is the ",i+1, "-th loop:")
        #sample_d_mp12(TT,AA,HH,uu,q, epsilon)
        #SS_id_transpose[i]=sample_d_mp12(TT_id,AA_id,HH_id,qIIn[i],q, epsilon)  #old code
        t1=timer()
        SS_id[i]=sample_d_mp12(TT_id,keys[0].augment(QQ_id),keys[2],q*unit_vector_i(i,n),q, epsilon) 
        t2=timer()
        print("                                      Time=", t2-t1)
        #print("ss_id=",SS_id_transpose[i])
        
    # Transpose to get SS_id in ZZ^{m+n*k x n}
    SS_id=SS_id.transpose()
    #private_key_id=SS_id
    #print("SS_id=")
    #print(SS_id)
    
    #assert AA_id*SS_id==qIIn
    #print("AA_id*SS_id=qIIn?",AA_id*SS_id==qIIn)
    
    #keys_id=(AA_id,SS_id)
    
    return (AA_id,SS_id)


#================================================
def idlrs_sign(public_parameters, master_public_key, message, event, ring_of_signers, real_signer):
    """
    
    * Input:
        - public_parameters: The set of public parameters. 
          Recall that (n, q, w, M, m, mt)=public_parameters
        - message: A message will be signed.
        - event: A event
        - ring_of_signers: A ring of signers:
        - real_signer: consists of the idenity and the private key of the real signer in the ring
    * Output:
        -
    """
    (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=public_parameters
    #print("sigma=",sigma)
     
    k=ceil(log(q,2))
    #print("m+n*k+n??=", m+n*k+n)
    
    ZZq=IntegerModRing(q)
    ZZ2q=IntegerModRing(2*q)
    
    (id_real_signer, SS_s)=real_signer #SS_s=private_key_of_real_signer
    
     # Number of signers in the ring
    l=len(ring_of_signers)
    
    #print("l=",l)
    
    for i in range(l):
        if id_real_signer==ring_of_signers[i]:
            order_real_signer=i
    #print("order_real_signer=",order_real_signer)
    
    
    
    IIn=identity_matrix(ZZ,n)
    qqIIn=q*identity_matrix(ZZ,n)
    #hash_to_a_matrix(string,n,m,q)
    KK=Matrix(ZZq,hash_to_a_matrix_generator_version(event,n, m+n*k, q))
    EE=KK*SS_s
    KK_hat=(2*KK).augment(-2*EE+qqIIn,subdivide=True)
    #print("KK_hat=")
    #print(KK_hat)
    
    SS_s_hat=SS_s.stack(IIn,subdivide=True)
    #print("SS_hat=")
    #print(SS_hat)
    
    #print("KK_hat*SS_hat=qqIIn??", KK_hat*SS_s_hat==qqIIn )
    
    AA_id_hat_list=[]
    OOn=zero_matrix(ZZ2q,n)
    for i in range(l):
        AA_i=public_key_for_each_identity(public_parameters, master_public_key, ring_of_signers[i])
        #print("AA_i=")
        #print(AA_i)
        AA_i_hat=AA_i.augment(OOn,subdivide=True)
        #print("AA_i_hat=")
        #print(AA_i_hat)
        AA_id_hat_list.append(AA_i_hat)
    
    #AA_s_hat=AA_id_hat_list[order_real_signer]   # The public key of the real signer augmented by zero matrix
    #print("AA_s_hat=")
    #print(AA_s_hat)
    

    # center vector in Gaussian distribution
    cc=zero_vector(ZZ,m+n*k+n)
    
    count=0
    
    #print("begin rejection sampling")
    #cc_list=[0]*l
    
    while True:
        t11=timer()
        count=count+1
        #print("                           Number of rejection sampling repetition in idlrs.sign: ",count)
        #print("inside rejection sampling")
        
        #string_list=[0]*l
        
    
  
        yy=sample_d(ZZ**(m+n*k+n),sigma,cc)  #sample_d(BB,s,cc)
       
        #print("yy=",yy)

        AA_s_hat_yy= AA_id_hat_list[order_real_signer]*yy
        KK_hat_yy=KK_hat*yy

        string_s=str(AA_s_hat_yy)+str(KK_hat_yy)+str(KK_hat)+str(ring_of_signers)+str(message)+str(event)+str(EE)
        #print("input_string=", input_string)
        del KK_hat_yy
        del AA_s_hat_yy

        #string_list[order_real_signer]= string_s
        
        cc_s_plus_1=hash_to_a_ball(string_s,n,w) #hash_to_a_ball(string,n,w)
        del string_s
        #cc_list[(order_real_signer+1)%l]=cc_s_plus_1

        #print("cc_s_plus_1=", cc_s_plus_1)

        #zz_list=[zero_vector(ZZ,m+n*k+n)]*l
        zz_list=[0]*l
        
        #print("zz_list=",zz_list)
        # From s+1 to l
        for i in range(order_real_signer+1,l):
            #t1=timer()
            zz_list[i]=sample_d(ZZ**(m+n*k+n),sigma,cc)  #sample_d(BB,sigma,cc)
            #t2=timer()
            #print("                            Sign: time for part 1:", t2-t1)
            #print("len(zz_i)=",len(zz_i))
            #zz_list[i]=zz_i
            AA_i_hat_zz=AA_id_hat_list[i]*zz_list[i]+q*cc_s_plus_1
            KK_hat_zz=KK_hat*zz_list[i]+q*cc_s_plus_1
            
            string_i=str(AA_i_hat_zz)+str(KK_hat_zz)+str(KK_hat)+str(ring_of_signers)+str(message)+str(event)+str(EE)
            
            del KK_hat_zz
            del AA_i_hat_zz
        
            cc_s_plus_1=hash_to_a_ball(string_i,n,w)
            # Save cc_1=H_2(AA_hat)
            #if i==l-1:
            del string_i
    
        cc_1=cc_s_plus_1
                
                # or cc_1=cc_list[0]
                #print("cc_1=",cc_1)
            #string_list[i]=str_i
            
            #cc_list[(i+1)%l]=cc_s_plus_1

        #print("zz_list_new", zz_list)

        # The output cc_s_plus_1 going out of the first for loop will be use in the second for loop.
        # It corresponds to cc_l in the paper
        for i in range(order_real_signer):
            #t1=timer()
            zz_list[i]=sample_d(ZZ**(m+n*k+n),sigma,cc)  #sample_d(BB,sigma,cc)
            #t2=timer()
            #print("                             Sign: time for part 2:", t2-t1)
            #print("zz_i=",zz_i)
            #zz_list[i]=zz_i
            AA_i_hat_zz=AA_id_hat_list[i]*zz_list[i]+q*cc_s_plus_1
            KK_hat_zz=KK_hat*zz_list[i]+q*cc_s_plus_1
            str_i=str(AA_i_hat_zz)+str(KK_hat_zz)+str(KK_hat)+str(ring_of_signers)+str(message)+str(event)+str(EE)
            
            del KK_hat_zz
            del AA_i_hat_zz
            
            cc_s_plus_1=hash_to_a_ball(str_i,n,w)
            
            #string_list[i]=str_i
            
            del str_i
            #cc_list[(i+1)%l]=cc_s_plus_1
            
        
        #print("zz_list_new", zz_list)
        cc_s=cc_s_plus_1   # Here we denote s=order_real_signer

        #Choose a random bit b<---{0,1}
        b=randint(0,1)
        #print("b=",b)

        
        xx=(-1)**b*SS_s_hat*cc_s
        zz_list[order_real_signer]=(xx+yy)%(2*q)
        
        #sigma_new=12*xx.norm()
        #M=exp(1+1/288)
        #print("alpha=", RR(alpha))
        #print("M=", RR(M))
        #print("sigma=",RR(sigma))
        #print("sigma_new=",RR(sigma_new))
        #if sigma_new >= sigma:
            #print("sigma_new >= sigma") 
        #u=uniform(0, 1)
        
        if count==4:
            print("                           Number of rejection sampling repetition in idlrs.sign: ",count)
            print("                          ==> Please choose different public parameters")
            break
            
        u=uniform(0, 1)
        if gauss_function_vector(zz_list[order_real_signer], cc, sigma)>=u*M*gauss_function_vector(zz_list[order_real_signer], xx, sigma):
            print("                           Number of rejection sampling repetition in idlrs.sign: ",count)
            print("                          ==> AWESOME!!! Rejection sampling worked well!")
            break
    #print("finish rejection sampling")
    #print("count=",count)

    # After rejection sampling
    #zz_list[order_real_signer]=zz_s
    
    #signature=(ring_of_signers, zz_list, cc_1, EE)
        t12=timer()
        print("                      time for each WHILE loop", t12-t11)
    
    
    #end while
    del KK_hat
    
    return (ring_of_signers, zz_list, cc_1, EE)

#========================================

def idlrs_verify(public_parameters, master_public_key,  message, event, signature):
    """
    """
    (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma) = public_parameters
    
    k=ceil(log(q,2))
    
    ZZq=IntegerModRing(q)
    ZZ2q=IntegerModRing(2*q)
    (ring_of_signers, zz_list, cc_1, EE)=signature
    #print("signature=", signature)
    
    #event="today"
    
    
    # STEP 1
    l=len(ring_of_signers)
    Lambda=1.3*sigma*sqrt(m+n*k+n)
    for i in range(l):
        if zz_list[i].norm()>Lambda:
            #print("zz_list[",i,"].norm()>Lambda")
            print("                                    The signature is INVALID because its element does not belong to the domain")
            return 0
    
    # STEP 2
    IIn=identity_matrix(ZZ,n)
    qqIIn=q*identity_matrix(ZZ,n)
    #hash_to_a_matrix(string,n,m,q)
    KK=Matrix(ZZq,hash_to_a_matrix_generator_version(event,n, m+n*k, q))
    KK_hat=(2*KK).augment(-2*EE+qqIIn,subdivide=True)
    #print("KK_hat=")
    #print(KK_hat)
    
    OOn=zero_matrix(ZZ2q,n)
    
    # STEP 3
    cc_i_plus_1=cc_1
    
    #string_list_new=[0]*l
    
    cc_list_new=[0]*l
    
    for i in range(l):
        AA_i=public_key_for_each_identity(public_parameters, master_public_key, ring_of_signers[i])
        
        #print("AA_i.nrows()=", AA_i.nrows())
        
        AA_i_hat=AA_i.augment(OOn,subdivide=True)
        #print("AA_i_hat=")
        #print(AA_i_hat)
        #AA_id_hat_list.append(AA_i_hat)
        
        zz_i=zz_list[i]
        
        #print("len(zz_i)=",len(zz_i))
        #print("AA_i_hat.nrows()=",AA_i_hat.nrows())
        #print("AA_i_hat.ncols()=",AA_i_hat.ncols())
        
        #print("len(cc_i_plus_1)=",len(cc_i_plus_1))
        
        AA_i_hat_zz=AA_i_hat*zz_i+q*cc_i_plus_1
        
        
        KK_hat_zz=KK_hat*zz_i+q*cc_i_plus_1
        str_i=str(AA_i_hat_zz)+str(KK_hat_zz)+str(KK_hat)+str(ring_of_signers)+str(message)+str(event)+str(EE)
        cc_i_plus_1=hash_to_a_ball(str_i,n,w)
        
        cc_list_new[(i+1)%l]=cc_i_plus_1
        #string_list_new[i]=str_i
        #print("str_i=",str_i)
    #print("cc_1=")
    #print(cc_1)
    
    
    #print("string_list_new=string_list???", string_list_new==string_list)
    
    #for i in range(l):
        #print("string_list_new[",i, "]=string_list[",i,"]???", string_list_new[i]==string_list[i])
        
    #for i in range(l):
        #print("cc_list_new[",i, "]=cc_list[",i,"]???", cc_list_new[i]==cc_list[i])
    
    #print("cc_i_plus_1=")
    #print(cc_i_plus_1)
    
    if cc_1==cc_list_new[0]:
        print("                                         The signature is VALID")
        return 1
    else: 
        print("                                         The signature is INVALID")
        return 0
#====================================================
def idlrs_link(signature_1, signature_2):
    """
    """
    
    (ring_of_signers_1, zz_list_1, cc_1_1, EE_1)=signature_1
    (ring_of_signers_2, zz_list_2, cc_1_2, EE_2)=signature_2
    if EE_1==EE_2:
        print("                          Two signarutres are linked")
        return 1
    else:
        print("                          Two signarutres are unlinked")
        return 0

#======================================






# ALGORITHMS FOR GETTING SIZE

def idlrs_get_public_key_size(AA):
    """
    Compute the size (in bytes) of the private key for identity id
    """
    bit_size= 0
    
    n=AA.nrows()
    m=AA.ncols()
    

    
    for i in range(n):
        for j in range(m):
            bit_size += int(AA[i,j]).bit_length()
    
    return int(int(bit_size)/(8*1024))


#=======================================
def idlrs_get_secret_key_size(TT):
    """
    Compute the size (in bytes) of the private key for identity id
    """
    bit_size= 0
    
    n=TT.nrows()
    m=TT.ncols()
    

    
    for i in range(n):
        for j in range(m):
            bit_size += int(TT[i,j]).bit_length()
    
    return int(int(bit_size)/(8*1024))


#========================================

def idlrs_get_private_key_size(SS_id):
    """
    Compute the size (in bytes) of the private key for identity id
    """
    bit_size= 0
    
    n=SS_id.nrows()
    m=SS_id.ncols()
    

    
    for i in range(n):
        for j in range(m):
            bit_size += int(SS_id[i,j]).bit_length()
    
    return int(int(bit_size)/(8*1024))


#=========================
def idlrs_get_signature_size(signature):
    """
    """
    bit_size=0
    (ring_of_signers, zz_list, cc_1, EE)=signature
    
    # bit length of zz_list
    for i in range(len(zz_list)):
        for j in range(len(zz_list[i])):
            bit_size +=int(zz_list[i][j]).bit_length()
    
    # bit length of cc_1
    for i in range(len(cc_1)):
        bit_size+=int(cc_1[i]).bit_length()
    
    # bit length of EE
    n=EE.nrows()
    m=EE.ncols()
    for i in range(n):
        for j in range(m):
            bit_size += int(EE[i,j]).bit_length()
    
    return int(int(bit_size)/(8*1024))


#=========================================

def output_avg_time_and_size_1(public_parameters_list, ring_of_signers_list, event, message, message_2, number_of_runs): 
    """
    """
    
    
    
    # List of average times
    avg_keygen_time_list = []
    avg_extract_time_list = []
    avg_sign_time_list = []
    avg_verify_time_list = []
    avg_link_time_list = []
    
    # List of average sizes
    avg_size_public_key_list=[]
    avg_size_secret_key_list=[]
    avg_size_private_key_list=[]
    avg_size_signature_list = []
    
    # number of signers list
    number_of_signers_list=[]
    
    # for a list of public_paramters
    for i in range(len(public_parameters_list)):
        public_parameters = public_parameters_list[i]
        print(1, "/ We are doing on the ", 1 , "-th public parameters (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=", public_parameters)
        
        for ring_of_signers in ring_of_signers_list:

            # mumber of ring signers
            number_of_signers = len(ring_of_signers)
            print("   The ring consists of ", number_of_signers, "signers")
            number_of_signers_list.append(number_of_signers)

            # Time lists
            keygen_time_list = []
            extract_time_list = []
            sign_time_list = []
            verify_time_list = []
            link_time_list = []

            # Size lists
            public_key_size_list = []
            secret_key_size_list = []
            private_key_size_list = []
            signature_size_list = []


            # Parse public parameters
            #(n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma)=public_parameters


            for j in range (0, number_of_runs):
                
                print("     -------The ",j+1, "-th run:")

                # Choose the real signer in the ring-of-signers for this run
                #l=int(len(ring_of_signers))
                order_of_real_signer = randint(0, number_of_signers-1) # the order of the real signer in the ring
                identity=ring_of_signers[order_of_real_signer] # identity of the real signer

                # Start

                # For keygen
                    # time
                #start1=timer()
                print("        ------------------Idlrs.Keygen starts")
                t1 = cputime()
                keys=idlrs_keygen(public_parameters) #(master_public_key, master_secret_key, tag_matrix)=keys
                keygen_time_list.append(cputime(t1))
                #end1=timer()
                #print("---------The running time is", end1-start1 , " sec")
                (master_public_key, master_secret_key, tag_matrix)=keys


                    # size
                        # Master public key size
                public_key_size  = idlrs_get_public_key_size(master_public_key) 
                print("              ----------------- public key size=",public_key_size, "bytes")
                         # Master secret key size
                size_secret_key  = idlrs_get_secret_key_size(master_secret_key) 
                
                print("              ----------------- secret key size=",secret_key_size, "bytes")

                print("         ------------------Idlrs.Keygen done", ", runtime=", cputime(t1), " sec")

                
                public_key_size_list.append(public_key_size)
                secret_key_size_list.append(secret_key_size)
                
                
                
                # test with the identity of the real signer
                
                # for extract
                print("        ------------------Idlrs.Extract starts")
                t2 = cputime()
                #start2=timer()
                keys_id=idlrs_extract(public_parameters, keys, identity)
                #end2=timer()
                extract_time_list.append(cputime(t2))
                #print("---------The running time is", end2-start2 , " sec")
                #(AA_s,SS_s)=keys_id

                #private_key_real_signer=keys_id[1]#SS_s

                real_signer=(ring_of_signers[order_of_real_signer], keys_id[1])
                

                     # Private key size
                private_key_size = idlrs_get_private_key_size(keys_id[1]) 
                private_key_size_list.append(private_key_size)
                print("               ------------- private key size=",size_private_key , "bytes")
                print("          ------------------Idlrs.Extract done,",  "runtime=", cputime(t2), " sec")
                
                # for Sign
                print("         ------------------Idlrs.Sign starts")
                t3 = cputime()
                signature=idlrs_sign(public_parameters, master_public_key, message, event, ring_of_signers, real_signer)
                sign_time_list.append(cputime(t3))
                
                (ring_of_signers, zz_list, cc_1, EE)=signature

            
                     # Signature size
                size_signature  = idlrs_get_signature_size(signature)
                size_signature_list.append(size_signature)
                print("                 --------- signature size=",size_signature , "bytes")
                print("        ------------------Idlrs.Sign done,",  "runtime=", cputime(t3) , " sec")
                
                
                # for Verify
                print("        ------------------Idlrs.Verify starts")
                t4 = cputime()
                idlrs_verify(public_parameters, master_public_key, message, event, signature)
                verify_time_list.append(cputime(t4))
                
                print("         ------------------Idlrs.Idlrs.Verify done,",  "runtime=", cputime(t4),  " sec")
                
                signature_2=idlrs_sign(public_parameters, master_public_key, message_2, event, ring_of_signers, real_signer)

                # for Link
                print("        ------------------Idlrs.Link starts")
                t5 = cputime()
                idlrs_link(signature, signature_2)
                link_time_list.append(cputime(t5))
                print("        ------------------Idlrs.Link done,",  "runtime=", cputime(t5) , " sec")
                
                print("     -------done for the ",j+1, "-th run")

            # end the third for loop  

            #========================= AVERAGE TIMES=========
            # Compute the average of times
            avg_keygen_time = statistics.mean(keygen_time_list)
            avg_extract_time = statistics.mean(extract_time_list)
            avg_sign_time = statistics.mean(sign_time_list)
            avg_verify_time = statistics.mean(verify_time_list)
            avg_link_time = statistics.mean(link_time_list)

            # add to the file
            #print("     Create file idlrs_time.txt to save the following infomation")
            #print("     (public_parameters_type, number_of_signers, avg_keygen_time, avg_extract_time, avg_sign_time, avg_verify_time, avg_link_time)")
            #g_time = open("idlrs_time_pp2.txt", "a")
            #g_time_info = str(0) + ',' +  str(number_of_signers)+ ',' +  str(avg_keygen_time) + ',' +  str(avg_extract_time) + ',' + str(avg_sign_time) + ','+ str(avg_verify_time) + '\n'
            #g_time.write(g_time_info)
            #g_time.close()

            # print to see
            print("          public_parameter[",i,"]=", public_parameters)
            print("          The ring has", number_of_signers, "signers")
            print("          Avg keygen time (s):", avg_keygen_time)
            print("          Avg extract time (s):", avg_extract_time)
            print("          Avg sign time (s):", avg_sign_time)
            print("          Avg verify time (s):", avg_verify_time)
            print("          Avg link time (s):", avg_link_time)
        


            #========================= AVERAGE SIZES=========
            # Compute the average of sizes
            avg_size_public_key = ceil(statistics.mean(size_public_key_list))
            avg_size_secret_key = ceil(statistics.mean(size_secret_key_list))
            avg_size_private_key = ceil(statistics.mean(size_private_key_list))
            avg_size_signature = ceil(statistics.mean(size_signature_list))

            # add to the file
            #print("           Create file idlrs_size.txt to save the following infomation")
            #print("           (public_parameters_type, number_of_signers, avg_size_public_key, avg_size_secret_key, avg_size_private_key, avg_size_signature)")
            #f_size = open("idlrs_size_pp2.txt", "a")
            #f_size_info = str(0) + ',' + str(number_of_signers) + ',' + str(avg_size_public_key) + ',' +  str(avg_size_secret_key) + ',' + str(avg_size_private_key) + ','+ str(avg_size_signature) + '\n'
            #f_size.write(f_size_info)
            #f_size.close()

            # print to see
            print("            Avg size of public_key (bytes):", avg_size_public_key)
            print("            Avg size of secret_key (bytes):", avg_size_secret_key)
            print("            Avg size of private_key (bytes):", avg_size_private_key)
            print("            Avg size of signature (bytes):", avg_size_signature)
            
            print("==================")
            print("SUMMARY (AVERAGE):")
            
            print("number_of_signers, avg_keygen_time, avg_extract_time, avg_sign_time, avg_verify_time, avg_link_time:")
            print(number_of_signers, ",", avg_keygen_time, ",", avg_extract_time, ",", avg_sign_time,",", avg_verify_time, ",", avg_link_time)
            
            print("        -------------")
            print("number_of_signers, avg_size_public_key, avg_size_secret_key, avg_size_private_key, avg_size_signature: ")
            print(number_of_signers, ",", avg_size_public_key,",", avg_size_secret_key, ",", avg_size_private_key,",", avg_size_signature)
            print("==================")
            
            print("    ------done for the case that the ring has", number_of_signers, "signers./.")
            

        # end the second for loop
        print("done for the ",1,"-th public parameters./.")
    # end the first for loop
    
  
    return 1 








#===========================================================
def output_avg_time_and_size(public_parameters, identity, ring_of_signers_list, event, message, message_2, public_key_title, secret_key_title, private_key_title): 
    """
    """
    

    # List of average times
    #avg_keygen_time_list = []
    #avg_extract_time_list = []
    #avg_sign_time_list = []
    #avg_verify_time_list = []
    #avg_link_time_list = []
    
    # List of average sizes
    #avg_size_public_key_list=[]
    #avg_size_secret_key_list=[]
    #avg_size_private_key_list=[]
    #avg_size_signature_list = []
    
    # number of signers list
    #number_of_signers_list=[]

    
    print("*** We are doing on the public parameters (n, q, mt, w, m, M, sigma_1, sigma_2, sigma_3, sigma):")
    print(public_parameters)
    
    #for ring_of_signers in ring_of_signers_list:
        # mumber of ring signers
    #number_of_signers = len(ring_of_signers)
   # print("    ** The ring consists of", number_of_signers, "signers")
    
        # Time lists
    #keygen_time_list = []
    #extract_time_list = []
    #sign_time_list = []
    #verify_time_list = []
    #link_time_list = []

        # Size lists
    #public_key_size_list = []
    #secret_key_size_list = []
    #private_key_size_list = []
    #signature_size_list = []
            
        # Choose the real signer in the ring-of-signers for this run
        #l=int(len(ring_of_signers))
    order_of_real_signer = 4 #randint(0, number_of_signers-1) # the order of the real signer in the ring
        #identity=ring_of_signers[order_of_real_signer] # identity of the real signer

        # Start
    #for run in range(number_of_runs):
        #print("        * The", run+1, "-th run:")
            # For keygen
    print("             1/ Idlrs.Keygen starts------------------")
    t1 =timer()
    (master_public_key, master_secret_key, tag_matrix)=idlrs_keygen(public_parameters) 
    t2=timer()

            
    keygen_time = t2-t1
                                # keygen time
    print("                    1.1./ Idlrs.Keygen time is:   ", keygen_time, "(s)")
                                # Master public key size
    public_key_size  = idlrs_get_public_key_size(master_public_key) 
    print("                    1.2./ public key size:        ", public_key_size , "(kB)")
                                 # Master secret key size
    secret_key_size  = idlrs_get_secret_key_size(master_secret_key) 
    print("                    1.3./ Master secret key size: ", secret_key_size , "(kB)")
    print("            ==> Idlrs.Keygen is done!")

    #keygen_time_list.append(keygen_time)
    #public_key_size_list.append(public_key_size)
    #secret_key_size_list.append(secret_key_size)
            
            # save the public key
    f_pk = open(public_key_title, "a")
    f_pk.write(str(master_public_key)+ "\n")
    f_pk.write("\n")
    f_pk.close()
            
            
            # save the secret key
    f_sk = open(secret_key_title, "a")
    f_sk.write(str(master_secret_key)+"\n")
    f_sk.write("\n")
    f_sk.close()
            
    
                        # test with the identity of the real signer

                        # for extract
    print("             2/ Idlrs.Extract starts------------------")
    t1 = timer()
    keys_id=idlrs_extract(public_parameters, (master_public_key, master_secret_key, tag_matrix), identity)
    t2 = timer()
    extract_time=t2-t1
                                    # time
    print("                     2.1./ Idlrs.Extract time is: ", extract_time , " (s)")                
                                   # Private key size
    private_key_size = idlrs_get_private_key_size(keys_id[1])       
    print("                     2.2./ private key size:      ", private_key_size , "(kB)")
    print("            ==> Idlrs.Extract is done!")

    #extract_time_list.append(extract_time)
    #private_key_size_list.append(private_key_size)
              
            # save the private key
    f_prk = open(private_key_title, "a")
    f_prk.write(str(keys_id[1])+"\n")
    f_prk.write("\n")
    f_prk.close()
    
    # save key generation time and extraction time
    keygen_extract_time = open("keygen_extract_time.txt", "a")
    keygen_extract_time_info = str(public_parameters[0]) + ',' +  str(keygen_time) + ',' +  str(extract_time) + '\n'
    keygen_extract_time.write(keygen_extract_time_info)
    keygen_extract_time.close()
    
    #save public key size, master secret key size and private key size

    pk_msk_prk_size = open("pk_msk_prk_size.txt", "a")
    pk_msk_prk_size_info = str(public_parameters[0]) + ',' + str(public_key_size) + ',' +  str(secret_key_size) + ',' + str(private_key_size) + '\n'
    pk_msk_prk_size.write(pk_msk_prk_size_info)
    pk_msk_prk_size.close()
    
           
    print("===============================================================")
    print("============================ SUMMARY for KEYGEN and EXTRACT ==========================")

    print("TIMES (in seconds):")

    print("n, keygen_time, extract_time:")

    print("and ")

    print("SIZES (in kB):")
    print("n, public_key_size, secret_key_size, private_key_size: ")
    print(public_parameters[0], ",",  keygen_time, ",", extract_time)
    print(public_parameters[0], ",", public_key_size,",", secret_key_size, ",", private_key_size)
    print("=================================================================")
    print("=========================== END SUMMARY for KEYGEN and EXTRACT ========================")
   
    

            
            
    for ring_of_signers in ring_of_signers_list:
        # mumber of ring signers
        number_of_signers = len(ring_of_signers)
        print("================================================")
        print("    ** The ring consists of", number_of_signers, "signers")
        print("================================================")
        
                        # for Sign
        real_signer=(ring_of_signers[order_of_real_signer], keys_id[1])
        print("              3/ Idlrs.Sign starts---------------------")
        t1=timer()
        signature=idlrs_sign(public_parameters, master_public_key, message, event, ring_of_signers, real_signer)
        t2=timer()
        sign_time=t2-t1
        print("                     3.1./ Idlrs.Sign time is:     ", sign_time , " (s)")
                             # Signature size
        signature_size  = idlrs_get_signature_size(signature)
        print("                     3.2./ signature size:         ", signature_size , "(kB)")

        print("              ==> Idlrs.Sign is done!")
        #sign_time_list.append(sign_time)
        #signature_size_list.append(signature_size)        



                        # for Verify
        print("              4/ Idlrs.Verify starts--------")
        t1 = timer()
        idlrs_verify(public_parameters, master_public_key, message, event, signature)
        t2=timer()
        verify_time = t2-t1

        print("                     4.1./ Idlrs.Verify time is:   ", verify_time,  " secs")

        #verify_time_list.append(verify_time)

        
        print("              ==> Idlrs.Verify is done!")

                        # for Link
        print("              5/ Idlrs.Link starts-----------------------")
        print("                   Sign on the second message for the idlrs.link algorithm")
        signature_2=idlrs_sign(public_parameters, master_public_key, message_2, event, ring_of_signers, real_signer)
            
        t1 = timer()
        idlrs_link(signature, signature_2)
        t2=timer()
        link_time=t2-t1
        print("                     5.1./ Idlrs.Link time is:    ",  link_time , " secs")
        print("              ==> Idlrs.Link is done!")
        #link_time_list.append(link_time)
        del signature
        del signature_2
            
        

        #========================= AVERAGE TIMES=========
        # Compute the average of times
        #avg_keygen_time = statistics.mean(keygen_time_list)
        #avg_extract_time = statistics.mean(extract_time_list)
        #avg_sign_time = statistics.mean(sign_time_list)
        #avg_verify_time = statistics.mean(verify_time_list)
        #avg_link_time = statistics.mean(link_time_list)
            
 
        # add to the file
        #print("     Create file idlrs_time.txt to save the following infomation")
        #print("     (public_parameters_type, number_of_signers, avg_keygen_time, avg_extract_time, avg_sign_time, avg_verify_time, avg_link_time)")
        sign_verify_link_time = open("sign_verify_link_time.txt", "a")
        sign_verify_link_time_info = str(public_parameters[0]) + ',' +  str(number_of_signers)+ ',' +  str(sign_time) + ','+ str(verify_time) + ','+ str(link_time)+ '\n'
        sign_verify_link_time.write(sign_verify_link_time_info)
        sign_verify_link_time.close()

        #========================= AVERAGE SIZES=========
        # Compute the average of sizes
        #avg_public_key_size = ceil(statistics.mean(public_key_size_list))
        #avg_secret_key_size = ceil(statistics.mean(secret_key_size_list))
        #avg_private_key_size = ceil(statistics.mean(private_key_size_list))
        #avg_signature_size = ceil(statistics.mean(signature_size_list))

        # add to the file
        #print("           Create file idlrs_size.txt to save the following infomation")
        #print("           (public_parameters_type, number_of_signers, avg_size_public_key, avg_size_secret_key, avg_size_private_key, avg_size_signature)")
        sign_size = open("sign_size.txt", "a")
        sign_size_info = str(public_parameters[0]) + ',' + str(number_of_signers) + ',' + str(signature_size) + '\n'
        sign_size.write(sign_size_info)
        sign_size.close()

            
            
        print("===============================================================")
        print("============================ SUMMARY for", number_of_signers, "signers./. ==========================")

        print("TIMES (in seconds):")

        print("n, number_of_signers,  avg_sign_time, avg_verify_time, avg_link_time:")

        print("and ")

        print("SIZES (in kB):")
        print("n, number_of_signers, signature_size: ")
        print(public_parameters[0], ",", number_of_signers, ",",  sign_time,",", verify_time, ",", link_time)
        print(public_parameters[0], ",", number_of_signers, ",", signature_size)
        print("=================================================================")
        print("=========================== END SUMMARY for", number_of_signers, "signers./. ========================")
        print("Done for the case of",  number_of_signers, "signers./.")
    
    
    print("All rings have been done!")
    
    return 1 

#===========================================================


#===== INPUT=====================
#===== INPUT=====================
id1="ktxops0715g@gmail.com"
id2="zkl89pl23efg@hotmail.com"
id3="vc2367oxg@easymail.com"
id4="ttxwq41n na jh2@kmail.com"
id5="dgajdanf09g@edumail.com"
id6="jfafkljkf018@ctymail.com"
id7="hsajfajnfkn812@abmail.com"
id8="hfajfksnfk@xxy.com"
id9="iq712uiqrfnkhb@xxy.com"
id10="79uriqfhjj@goodmail.com"

id11="79ffhjfhgndngsgn@goodmail.com"
id12="79fhfdsfsdjshgburiqfhjj@goodmail.com"
id13="79uriqfgueyiqwoirehjj@goodmail.com"
id14="7jkfdskgdmail.com"
id15="79uriqfhsjgfkjngfdfjhfjfhdjmail.com"
id16="79uriqfhjj@goodmaijfhaddsagkdagl.com"
id17="79uriqfhjj@goodmail.comfusdgajhfgd"
id18="79uriqfhjj@goodmail.cgfjbafsjdsbom"
id19="79uriqfhjj@goodmail.cdhjasfhjom"
id20="79uriqfhjj@ggjahkfjdsjfdoodmail.com"

id21="abcdef1856g@mailmail.com"
id22="ktxops0715g@gmail.com"
id23="zkl89pl23efg@hotmail.com"
id24="vc2367oxg@easymail.com"
id25="ttxwq412@kmail.com"
id26="dgajdanf09g@edumail.com"
id27="jfafkljkf018@ctymail.com"
id28="hsajfajnfkn812@abmail.com"
id29="hfajfksnfk@xxy.com"
id30="iq712uiqrfnkhb@xxy.com"

id31="jkfjoi79uriqfhjj@goodmail.com"
id32="7kjkja9ffhjdgdhghjd98764743fhgndngsgn@goodmail.com"
id33="kafjafj79fhfdjdhadsjkdsfkj79834248sfsdjshgburiqfhjj@goodmail.com"
id34="iu79uriqfgu74393rhfjcjb eyiqwoirehjj@goodmail.com"
id35="ajbcc7jkfdskgdkfhncj9783793mail.com"
id36="da79uriq74382hfjbfjbfhsjgfkjngfdfjhfjfhdjmail.com"
id37="fafa79uridhjhkcjbam nmn qfhjj@goodmaijfhaddsagkdagl.com"
id38="7faf9uriqfhjj@fifjiu989880283goodmail.comfusdgajhfgd"
id39="faf79affauriqfhj573t214yhhj@goodmail.cgfjbafsjdsbom"
id40="745669uriqfhjyfuychjnjnj@goodmail.cdhjasfhjom"


ring_of_signers_1=[id1, id2, id3, id4, id5, id6, id7, id8, id9, id10]
ring_of_signers_2=[id1, id2, id3, id4, id5, id6, id7, id8, id9, id10, id11, id12, id13, id14, id15, id16, id17, id18, id19, id20]
ring_of_signers_3=[id1, id2, id3, id4, id5, id6, id7, id8, id9, id10, id11, id12, id13, id14, id15, id16, id17, id18, id19, id20, id21, id22, id23, id24, id25, id26, id27, id28, id29, id30]
ring_of_signers_4=[id1, id2, id3, id4, id5, id6, id7, id8, id9, id10, id11, id12, id13, id14, id15, id16, id17, id18, id19, id20, id21, id22, id23, id24, id25, id26, id27, id28, id29, id30, id31, id32, id33, id34, id35, id36, id37, id38, id39, id40]

ring_of_signers_list=[ring_of_signers_1, ring_of_signers_2, ring_of_signers_3, ring_of_signers_4]

message="We can do everything well. Do not worry about it guys!"
message_2="Lattice-based crypto is awesome!!!"
event="with the company ABC"


#public_parameters=(10,524288,190,5,380,3,4.48083023712027,204.917923135140,7197.66362631348,1.03105118283403e7)


public_parameters_10 = (10,524288,190,5,380,3,4.48083023712027,204.917923135140,7197.66362631348,5.03105118283403e8)
public_parameters_20=(20,8388608,460,6,920,3,4.48083023712027,778.301975076363,190598.865266524,9.62728584328202e9)
public_parameters_40 = (40,67108864,1040,11,2080,3,4.48083023712027,1165.22352070264,429061.131614986,8.11257427288557e10)
public_parameters_60 = (60,536870912,1740,15,3480,3,4.48083023712027,1504.24674659589,716451.780933469,3.21126195212814e11)
public_parameters_80 = (80,4294967296,2560,20,5120,3,4.48083023712027,1822.45271625383,1.05285730202738e6,2.87875508121890e12)
public_parameters_100 = (100,34359738368,3500,24,7000,3,4.48083023712027,2129.23955112039,1.43830770969715e6,5.51800764686501e13)
public_parameters_120 = (120,274877906944,4560,29,9120,3,4.48083023712027,2428.95592930687,1.87281689232807e6,9.90971039925680e14)

number_of_runs=10
    
#========================== RUNNNN===

#if __name__ == '__main__':
    #print("The main section has been started")
    #print("*********************************")
    #output_avg_time_and_size_1(public_parameters_list, ring_of_signers_list, event, message, message_2, number_of_runs)
public_key_title_10="public_key_10.txt"
public_key_title_20="public_key_20.txt"
public_key_title_40="public_key_40.txt"
public_key_title_60="public_key_60.txt"
public_key_title_80="public_key_80.txt"
public_key_title_100="public_key_100.txt"
public_key_title_120="public_key_120.txt"


secret_key_title_10="secret_key_10.txt"
secret_key_title_20="secret_key_20.txt"
secret_key_title_40="secret_key_40.txt"
secret_key_title_60="secret_key_60.txt"
secret_key_title_80="secret_key_80.txt"
secret_key_title_100="secret_key_100.txt"
secret_key_title_120="secret_key_120.txt"


private_key_title_10="private_key_10.txt"
private_key_title_20="private_key_20.txt"
private_key_title_40="private_key_40.txt"
private_key_title_60="private_key_60.txt"
private_key_title_80="private_key_80.txt"
private_key_title_100="private_key_100.txt"
private_key_title_120="private_key_120.txt"
    
if __name__ == '__main__':
    print("The main section has been started")
    print("*********************************")
  
    # TITLES for THE NAMEs of FILE OUTPUT
    title_10_time="times_for_n_10.txt"
    title_10_size="sizes_for_n_10.txt"
    
    title_20_time="times_for_n_20.txt"
    title_20_size="sizes_for_n_20.txt"
      
    title_40_time="times_for_n_40.txt"
    title_40_size="sizes_for_n_40.txt"
   
    title_60_time="times_for_n_60.txt"
    title_60_size="sizes_for_n_60.txt"
    
    title_80_time="times_for_n_80.txt"
    title_80_size="sizes_for_n_80.txt"
    
    title_100_time="times_for_n_100.txt"
    title_100_size="sizes_for_n_100.txt"
   
    
    title_120_time="times_for_n_120.txt"
    title_120_size="sizes_for_n_120.txt"
   
    identity=id5
    print("We run", number_of_runs, "times")
    for i in range(number_of_runs):
        print("the", i+1, "-th run:")
        #output_avg_time_and_size(public_parameters_10, identity, ring_of_signers_list, event, message, message_2,  public_key_title_10, secret_key_title_10, private_key_title_10)
        #output_avg_time_and_size(public_parameters_20, identity, ring_of_signers_list, event, message, message_2, public_key_title_20, secret_key_title_20, private_key_title_20)
        #output_avg_time_and_size(public_parameters_40, identity, ring_of_signers_list, event, message, message_2,  public_key_title_40, secret_key_title_40, private_key_title_40)
        #output_avg_time_and_size(public_parameters_60, identity, ring_of_signers_list, event, message, message_2,  public_key_title_60, secret_key_title_60, private_key_title_60)
        #output_avg_time_and_size(public_parameters_80, identity, ring_of_signers_list, event, message, message_2,  public_key_title_80, secret_key_title_80, private_key_title_80)
        #output_avg_time_and_size(public_parameters_100, identity, ring_of_signers_list, event, message, message_2,  public_key_title_100, secret_key_title_100,  private_key_title_100)
        #output_avg_time_and_size(public_parameters_120, identity, ring_of_signers_list, event, message, message_2, public_key_title_120, secret_key_title_120, private_key_title_120)

The main section has been started
*********************************
We run 1 times
the 1 -th run:
